In [ ]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [ ]:
!pip install -U pinecone_notebooks

In [ ]:
import os
from pinecone_notebooks.colab import Authenticate

if not os.environ.get("pcsk_346CtC_P7GPEDx1WxTNeaGgzVeMmDwBsdoFDFLTrTNuV3swxcGHPRT5dVxyNHQFcWHrvjM"):
    Authenticate()

In [ ]:
# Import the Pinecone library
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
api_key = os.environ.get("pcsk_346CtC_P7GPEDx1WxTNeaGgzVeMmDwBsdoFDFLTrTNuV3swxcGHPRT5dVxyNHQFcWHrvjM")
pc = Pinecone(api_key=api_key)

In [ ]:
!pip install --upgrade --quiet langchain pypdf sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
import os
from PyPDF2 import PdfReader

# Set the file path
pdf_path = Path("/budget_speech.pdf")

# Load the PDF
reader = PdfReader(str(pdf_path))

# Extract text
text = ""
for page in reader.pages:
    text += page.extract_text()

print(text[:1000])  # preview the first 1000 characters


GOVERNMENT OF INDIA
BUDGET 2023-2024
SPEECH
OF
NIRMALA SITHARAMAN
MINISTER OF FINANCE
February 1,  2023CONTENTS 
PART-A 
 Page No.  
 Introduction 1 
 Achievements since 2014: Leaving no one behind 2 
 Vision for Amrit Kaal  – an empowered and inclusive economy 3 
 Priorities of this Budget 5 
i. Inclusive Development  
ii. Reaching the Last Mile 
iii. Infrastructure and Investment 
iv. Unleashing the Potential 
v. Green Growth 
vi. Youth Power  
vii. Financial Sector  
 
 
 
 
 
 
 
 
 Fiscal Management 24 
PART B  
  
Indirect Taxes  27 
 Green Mobility  
 Electronics   
 Electrical   
 Chemicals and Petrochemicals   
 Marine products  
 Lab Grown Diamonds  
 Precious Metals  
 Metals  
 Compounded Rubber  
 Cigarettes  
  
Direct Taxes  30 
 MSMEs and Professionals   
 Cooperation  
 Start-Ups  
 Appeals  
 Better targeting of tax concessions  
 Rationalisation  
 Others  
 Personal Income Tax  
  
Annexures  35 
 Annexure to Part B of the Budget Speech 2023-

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

texts = text_splitter.split_text(text)
print(f"Split into {len(texts)} chunks")


Split into 226 chunks


In [ ]:
from sentence_transformers import SentenceTransformer

# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings
embeddings = embedder.encode(texts, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
from pinecone import Pinecone, ServerlessSpec

if 'budget-speech-latest-rag' not in pc.list_indexes().names():
      pc.create_index(
          name='budget-speech-latest-rag',
          dimension=384,
          metric='cosine',
          spec=ServerlessSpec(
              cloud='aws',
              region='us-east-1'
          )
      )

In [ ]:
index = pc.Index('budget-speech-latest-rag')
vectors = [(f"id-{i}", embeddings[i], {"text": texts[i]}) for i in range(len(texts))]
index.upsert(vectors, namespace="budget-speech-latest-rag")


{'upserted_count': 226}

In [ ]:
query = "explain the philosophy, Sabka Saath Sabka Vikas"
query_embedding = embedder.encode([query])[0]

# Search in Pinecone
results = index.query(vector=query_embedding.tolist(), top_k=1, include_metadata=True)

# Display retrieved texts
for match in results['matches']:
    print(match['metadata']['text'], '\n' + '-'*80)


15. The Government’s philosophy of Sabka Saath Sabka Vikas  has 
facilitated inclusive development covering in specific, farmers, women, 
youth, OBCs, Scheduled Castes, Scheduled Tribes, divyangjan and 
economically weaker sections, and overall priority for the underprivileged 
(vanchiton ko variyata ). There has also been a sustained focus on Jammu & 
Kashmir, Ladakh and the North-East. This Budget builds on those efforts.  
Agriculture and Cooperation 
--------------------------------------------------------------------------------
